### enhancement to the main notebook, analyzing only beneficiaries that are hospice candidates ###


In [4]:
import glob
%matplotlib inline
import sys
print(sys.version)
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)
import matplotlib.pyplot as plt


3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
1.16.4
0.24.2


In [5]:
def df_write_csv(df, path):
    
    df.to_csv(path_or_buf = path, index=False)

In [6]:
bene_df = pd.read_csv("../Working_Folder_VM/bene_cohort.csv")
bene_df.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,SP_STRKETIA,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR
0,000102649ED5601B,1931-10-01,NaN,2,1,0,49,290,12,12,...,1,0.0,0.0,0.0,360.0,90.0,0.0,860.0,170.0,0.0
1,0002278C944E240A,1936-09-01,NaN,2,1,Y,34,250,12,12,...,2,0.0,0.0,0.0,2590.0,1530.0,0.0,5000.0,1230.0,0.0
2,000330E625C93700,1933-09-01,NaN,2,1,0,4,420,12,12,...,2,0.0,0.0,0.0,0.0,0.0,0.0,270.0,150.0,0.0
3,000374D5E110EDA6,1936-08-01,NaN,2,1,0,5,90,12,12,...,2,0.0,0.0,0.0,60.0,30.0,0.0,1090.0,360.0,0.0
4,0003950E4B4FEC8D,1940-11-01,NaN,2,2,0,19,420,12,12,...,2,0.0,0.0,0.0,3690.0,420.0,0.0,3590.0,960.0,0.0


In [7]:
bene_df['BENE_COUNT_OCCURENCE'] = 1
bene_df['BENE_HOSPICE_CANDIDATE'] = np.where((bene_df['SP_ALZHDMTA'] == 1)
                                 |(bene_df['SP_CHF'] == 1)
                                 |(bene_df['SP_CHRNKIDN'] == 1)
                                 |(bene_df['SP_CNCR'] == 1)
                                 |(bene_df['SP_COPD'] == 1)
                                 |(bene_df['SP_ISCHMCHT'] == 1)
                                 |(bene_df['SP_STRKETIA'] == 1)
                                 ,1,0)

bene_df["BENE_BIRTH_DT"] = pd.to_datetime(bene_df["BENE_BIRTH_DT"], format="%Y-%m-%d")
bene_df["BENE_DEATH_DT"] = pd.to_datetime(bene_df["BENE_DEATH_DT"], format="%Y-%m-%d")

bene_df_hospice = bene_df[bene_df.BENE_HOSPICE_CANDIDATE == 1]
bene_df_hospice = bene_df_hospice[(bene_df_hospice.BENE_HI_CVRAGE_TOT_MONS == 12) & (bene_df_hospice.BENE_SMI_CVRAGE_TOT_MONS == 12)]



In [8]:
bene_df_hospice.dtypes

DESYNPUF_ID                         object
BENE_BIRTH_DT               datetime64[ns]
BENE_DEATH_DT               datetime64[ns]
BENE_SEX_IDENT_CD                    int64
BENE_RACE_CD                         int64
BENE_ESRD_IND                       object
SP_STATE_CODE                        int64
BENE_COUNTY_CD                       int64
BENE_HI_CVRAGE_TOT_MONS              int64
BENE_SMI_CVRAGE_TOT_MONS             int64
BENE_HMO_CVRAGE_TOT_MONS             int64
PLAN_CVRG_MOS_NUM                    int64
SP_ALZHDMTA                          int64
SP_CHF                               int64
SP_CHRNKIDN                          int64
SP_CNCR                              int64
SP_COPD                              int64
SP_DEPRESSN                          int64
SP_DIABETES                          int64
SP_ISCHMCHT                          int64
SP_OSTEOPRS                          int64
SP_RA_OA                             int64
SP_STRKETIA                          int64
MEDREIMB_IP

In [20]:
bene_hospice_aggr = (bene_df_hospice.groupby('DESYNPUF_ID').agg(
    {
     'BENE_BIRTH_DT':'max',
     'BENE_DEATH_DT':'max',
     'BENE_SEX_IDENT_CD':'max',
     'BENE_RACE_CD': 'max',
     'SP_STATE_CODE': 'max',
     'BENE_COUNTY_CD': 'max',
     'BENE_HI_CVRAGE_TOT_MONS':'max',
     'BENE_SMI_CVRAGE_TOT_MONS':'max',
     'BENE_COUNT_OCCURENCE':'sum',
     'BENE_HOSPICE_CANDIDATE': 'sum'
    }
    )
             .reset_index()
             .rename(columns={'BENE_BIRTH_DT':'BENE_BIRTH_DT_max',
                              'BENE_SEX_IDENT_CD':'BENE_SEX_IDENT_CD_max',
                              'BENE_RACE_CD': 'BENE_RACE_CD_max',
                              'SP_STATE_CODE': 'SP_STATE_CODE_max',
                              'BENE_HI_CVRAGE_TOT_MONS':'BENE_HI_CVRAGE_TOT_MONS_max',
                              'BENE_SMI_CVRAGE_TOT_MONS':'BENE_SMI_CVRAGE_TOT_MONS_max',
                              'BENE_COUNTY_CD':'BENE_COUNTY_CD_max',
                              'BENE_HOSPICE_CANDIDATE': 'BENE_HOSPICE_CANDIDATE_sum'
                             }))



In [21]:
#get carrier, inpatient and outpatient aggregates 
car_aggr = pd.read_csv("../Working_Folder_VM/car_cohort_aggr.csv")
inp_aggr = pd.read_csv("../Working_Folder_VM/inp_cohort_aggr.csv")
outp_aggr = pd.read_csv("../Working_Folder_VM/outp_cohort_aggr.csv")


In [22]:
#now merge beneficiary, carrier claims, inpatient and outpatient by left join 
bene_car_df = pd.merge(bene_hospice_aggr, car_aggr, on='DESYNPUF_ID', how = 'left')
bene_car_inp_df = pd.merge(bene_car_df, inp_aggr, on='DESYNPUF_ID', how = 'left')
bene_car_inp_outp_df = pd.merge(bene_car_inp_df, outp_aggr, on='DESYNPUF_ID', how = 'left')
#should've differentiated inpatient and outpatient columns while creating those dataframes. Ah well!
bene_car_inp_outp_df = bene_car_inp_outp_df.rename(columns= {'CLM_ID_count_x': 'car_CLM_ID_count',
                                                            'CLM_ID_count_y': 'inp_CLM_ID_count',
                                                            'CLM_PMT_AMT_sum_x': 'inp_CLM_PMT_AMT_sum',
                                                            'NCH_PRMRY_PYR_CLM_PD_AMT_sum_x': 'inp_NCH_PRMRY_PYR_CLM_PD_AMT_sum',
                                                            'CLM_ID_count': 'outp_CLM_ID_count',
                                                            'CLM_PMT_AMT_sum_y': 'outp_CLM_PMT_AMT_sum',
                                                            'NCH_PRMRY_PYR_CLM_PD_AMT_sum_y':'outp_NCH_PRMRY_PYR_CLM_PD_AMT_sum'})



In [24]:
bene_car_inp_outp_df.head()
print(bene_car_inp_outp_df.groupby('BENE_SEX_IDENT_CD_max').DESYNPUF_ID.count())
print(bene_car_inp_outp_df.groupby('BENE_RACE_CD_max').DESYNPUF_ID.count())


BENE_SEX_IDENT_CD_max
1    350009
2    465993
Name: DESYNPUF_ID, dtype: int64
BENE_RACE_CD_max
1    688522
2     80409
3     29826
5     17245
Name: DESYNPUF_ID, dtype: int64


In [25]:
#Now add state names
us_state_df = pd.read_csv("../Datasets/us_states_codes.csv")
us_state_df.head()
bene_car_inp_outp_state_df = pd.merge(bene_car_inp_outp_df, us_state_df, on='SP_STATE_CODE_max')

#Now fill NaNs with 0
values = {'car_CLM_ID_count':0, 'NCH_PMT_AMT_sum': 0, 'inp_CLM_ID_count': 0, 'inp_CLM_PMT_AMT_sum': 0, 'outp_CLM_ID_count': 0}
bene_car_inp_outp_state_df.fillna(value=values,inplace=True)

In [26]:
#Now add hospice indicator 
bene_car_inp_outp_state_df['HOSPICE_FLAG'] = np.where((bene_car_inp_outp_state_df['car_CLM_ID_count'] > 0)
                                 |(bene_car_inp_outp_state_df['inp_CLM_ID_count'] > 0)
                                 |(bene_car_inp_outp_state_df['outp_CLM_ID_count'] > 0)
                                 ,1,0)

In [27]:
# Total beneficiary and hospice beneficiary numbers by state
bene_country_group_by = (bene_car_inp_outp_state_df.groupby('SP_State_full_name').agg(
    {
     'DESYNPUF_ID':'count',
     'HOSPICE_FLAG':'sum'
    }
    )
             .reset_index()
             .rename(columns={'DESYNPUF_ID':'DESYNPUF_ID_count',
                              'HOSPICE_FLAG':'HOSPICE_FLAG_sum'
                             }))

bene_country_group_by['percent_hospice'] = bene_country_group_by['HOSPICE_FLAG_sum']/bene_country_group_by['DESYNPUF_ID_count']

df_write_csv(bene_country_group_by,"../Working_Folder_VM/bene_hospice_cohort_country_groupby.csv")

### Valerie exploration

In [28]:
bene_car_inp_outp_state_df.dtypes

DESYNPUF_ID                                  object
BENE_BIRTH_DT_max                    datetime64[ns]
BENE_DEATH_DT                        datetime64[ns]
BENE_SEX_IDENT_CD_max                         int64
BENE_RACE_CD_max                              int64
SP_STATE_CODE_max                             int64
BENE_COUNTY_CD_max                            int64
BENE_HI_CVRAGE_TOT_MONS_max                   int64
BENE_SMI_CVRAGE_TOT_MONS_max                  int64
BENE_COUNT_OCCURENCE                          int64
BENE_HOSPICE_CANDIDATE_sum                    int64
car_CLM_ID_count                            float64
NCH_PMT_AMT_sum                             float64
inp_CLM_ID_count                            float64
inp_CLM_PMT_AMT_sum                         float64
inp_NCH_PRMRY_PYR_CLM_PD_AMT_sum            float64
CLM_UTLZTN_DAY_CNT_sum                      float64
outp_CLM_ID_count                           float64
outp_CLM_PMT_AMT_sum                        float64
outp_NCH_PRM

In [29]:
print(pd.crosstab(bene_car_inp_outp_state_df.HOSPICE_FLAG, bene_car_inp_outp_state_df
                  .BENE_HOSPICE_CANDIDATE_sum))

BENE_HOSPICE_CANDIDATE_sum       1       2       3
HOSPICE_FLAG                                      
0                           176307  234577  369838
1                             2545    8288   24447


In [30]:
bene_car_inp_outp_state_df.HOSPICE_FLAG.describe()

count    816002.000000
mean          0.043235
std           0.203386
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: HOSPICE_FLAG, dtype: float64

In [31]:
bene_car_inp_outp_state_df.BENE_HOSPICE_CANDIDATE_sum.describe()

count    816002.000000
mean          2.264010
std           0.795406
min           1.000000
25%           2.000000
50%           2.000000
75%           3.000000
max           3.000000
Name: BENE_HOSPICE_CANDIDATE_sum, dtype: float64

In [32]:
bene_country_group_by.percent_hospice.describe()

count    52.000000
mean      0.041872
std       0.003519
min       0.033035
25%       0.039820
50%       0.042327
75%       0.044692
max       0.048046
Name: percent_hospice, dtype: float64

In [33]:
# Total beneficiary and hospice beneficiary numbers by gender
bene_gender_group_by = (bene_car_inp_outp_state_df.groupby('BENE_SEX_IDENT_CD_max').agg(
    {
     'DESYNPUF_ID':'count',
     'HOSPICE_FLAG':'sum'
    }
    )
             .reset_index()
             .rename(columns={'DESYNPUF_ID':'DESYNPUF_ID_count',
                              'HOSPICE_FLAG':'HOSPICE_FLAG_sum'
                             }))

bene_gender_group_by['percent_hospice'] = bene_gender_group_by['HOSPICE_FLAG_sum']/bene_gender_group_by['DESYNPUF_ID_count']

df_write_csv(bene_gender_group_by,"../Working_Folder_VM/bene_hospice_cohort_gender_groupby.csv")

In [34]:
# Total beneficiary and hospice beneficiary numbers by race
bene_race_group_by = (bene_car_inp_outp_state_df.groupby('BENE_RACE_CD_max').agg(
    {
     'DESYNPUF_ID':'count',
     'HOSPICE_FLAG':'sum'
    }
    )
             .reset_index()
             .rename(columns={'DESYNPUF_ID':'DESYNPUF_ID_count',
                              'HOSPICE_FLAG':'HOSPICE_FLAG_sum'
                             }))

bene_race_group_by['percent_hospice'] = bene_race_group_by['HOSPICE_FLAG_sum']/bene_race_group_by['DESYNPUF_ID_count']

df_write_csv(bene_race_group_by,"../Working_Folder_VM/bene_hospice_cohort_race_groupby.csv")